# Ollama Introduction: Getting Started with Local LLMs

Welcome to the world of local Large Language Models! In this notebook, you'll learn how to use Ollama to run powerful AI models directly on your computer.

## What You'll Learn

- How to install and set up Ollama
- Basic model management (downloading, listing, removing models)
- Making your first API calls
- Interactive examples with different models
- Parameter adjustment and experimentation

## Prerequisites

- Python 3.8 or higher
- At least 8GB of RAM (16GB recommended)
- Internet connection for initial model downloads

Let's get started!

## 1. Installation and Setup

First, let's check if Ollama is installed and install the required Python packages.

In [ ]:
# Install required packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = ['requests', 'ipywidgets', 'matplotlib', 'pandas']

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)
        print(f"✓ {package} installed successfully")

In [ ]:
# Import necessary libraries
import requests
import json
import time
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

## 2. Ollama Connection and Health Check

Let's create a simple class to interact with Ollama and check if it's running.

In [ ]:
class OllamaClient:
    def __init__(self, base_url="http://localhost:11434"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def is_running(self):
        """Check if Ollama server is running"""
        try:
            response = self.session.get(f"{self.base_url}/api/tags", timeout=5)
            return response.status_code == 200
        except requests.exceptions.RequestException:
            return False
    
    def list_models(self):
        """List all available models"""
        try:
            response = self.session.get(f"{self.base_url}/api/tags")
            if response.status_code == 200:
                return response.json().get('models', [])
            return []
        except requests.exceptions.RequestException as e:
            print(f"Error listing models: {e}")
            return []
    
    def pull_model(self, model_name, callback=None):
        """Download a model from Ollama registry"""
        try:
            response = self.session.post(
                f"{self.base_url}/api/pull",
                json={"name": model_name},
                stream=True
            )
            
            for line in response.iter_lines():
                if line:
                    data = json.loads(line.decode('utf-8'))
                    if callback:
                        callback(data)
                    if data.get('status') == 'success':
                        return True
            return True
        except Exception as e:
            print(f"Error pulling model: {e}")
            return False
    
    def generate(self, model, prompt, system=None, stream=False):
        """Generate text using specified model"""
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": stream
        }
        
        if system:
            payload["system"] = system
        
        try:
            response = self.session.post(
                f"{self.base_url}/api/generate",
                json=payload
            )
            
            if response.status_code == 200:
                return response.json()
            else:
                return {"error": f"HTTP {response.status_code}: {response.text}"}
        except Exception as e:
            return {"error": str(e)}

# Initialize Ollama client
ollama = OllamaClient()

# Check if Ollama is running
if ollama.is_running():
    print("✓ Ollama is running and accessible!")
    models = ollama.list_models()
    print(f"Found {len(models)} models installed")
else:
    print("❌ Ollama is not running or not accessible")
    print("Please make sure Ollama is installed and running.")
    print("Visit: https://ollama.ai for installation instructions")

## 3. Model Management

Let's explore the models available and learn how to download new ones.

In [ ]:
def display_models():
    """Display available models in a nice format"""
    models = ollama.list_models()
    
    if not models:
        print("No models found. Let's download a small model to get started!")
        return []
    
    print("Available Models:")
    print("-" * 60)
    
    model_data = []
    for model in models:
        name = model.get('name', 'Unknown')
        size = model.get('size', 0)
        size_gb = size / (1024**3) if size > 0 else 0
        modified = model.get('modified_at', '')
        
        model_data.append({
            'Name': name,
            'Size (GB)': f"{size_gb:.2f}",
            'Modified': modified[:10] if modified else 'Unknown'
        })
        
        print(f"📦 {name}")
        print(f"   Size: {size_gb:.2f} GB")
        print(f"   Modified: {modified[:10] if modified else 'Unknown'}")
        print()
    
    return model_data

model_list = display_models()

### Interactive Model Download

If you don't have any models, let's download a small one to get started. We'll use `llama2:7b-chat` as it's a good balance of capability and size.

In [ ]:
# Interactive model downloader
def download_model_with_progress(model_name):
    """Download model with progress display"""
    progress_widget = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Downloading:',
        bar_style='info',
        style={'bar_color': 'blue'},
        orientation='horizontal'
    )
    
    status_label = widgets.Label(value="Starting download...")
    display(progress_widget, status_label)
    
    def update_progress(data):
        status = data.get('status', '')
        if 'completed' in data and 'total' in data:
            completed = data['completed']
            total = data['total']
            if total > 0:
                progress = int((completed / total) * 100)
                progress_widget.value = progress
                status_label.value = f"{status}: {progress}% ({completed}/{total} bytes)"
        else:
            status_label.value = status
    
    success = ollama.pull_model(model_name, callback=update_progress)
    
    if success:
        progress_widget.value = 100
        progress_widget.bar_style = 'success'
        status_label.value = f"✓ Successfully downloaded {model_name}!"
    else:
        progress_widget.bar_style = 'danger'
        status_label.value = f"❌ Failed to download {model_name}"
    
    return success

# Create download interface
model_dropdown = widgets.Dropdown(
    options=['llama2:7b-chat', 'codellama:7b', 'mistral:7b', 'phi:2.7b'],
    value='llama2:7b-chat',
    description='Model:',
)

download_button = widgets.Button(
    description='Download Model',
    button_style='primary',
    icon='download'
)

def on_download_click(b):
    model_name = model_dropdown.value
    print(f"Downloading {model_name}...")
    download_model_with_progress(model_name)

download_button.on_click(on_download_click)

print("Select a model to download:")
display(widgets.HBox([model_dropdown, download_button]))

## 4. Your First Ollama Conversation

Now let's have our first conversation with a local LLM!

In [ ]:
def simple_chat(model_name, prompt):
    """Simple chat function with timing"""
    start_time = time.time()
    
    print(f"🤖 Using model: {model_name}")
    print(f"👤 You: {prompt}")
    print("🤖 Assistant: ", end="")
    
    response = ollama.generate(model_name, prompt)
    
    end_time = time.time()
    
    if 'error' in response:
        print(f"❌ Error: {response['error']}")
        return None
    
    assistant_response = response.get('response', 'No response received')
    print(assistant_response)
    
    # Display timing information
    duration = end_time - start_time
    print(f"\n⏱️ Response time: {duration:.2f} seconds")
    
    return {
        'prompt': prompt,
        'response': assistant_response,
        'duration': duration,
        'model': model_name
    }

# Test with a simple prompt
models = ollama.list_models()
if models:
    test_model = models[0]['name']
    result = simple_chat(test_model, "Hello! Can you introduce yourself?")
else:
    print("Please download a model first using the interface above.")

## 5. Interactive Chat Interface

Let's create an interactive chat interface where you can experiment with different prompts and parameters.

In [ ]:
# Interactive chat interface
class InteractiveChat:
    def __init__(self):
        self.conversation_history = []
        self.setup_widgets()
    
    def setup_widgets(self):
        # Get available models
        models = ollama.list_models()
        model_names = [model['name'] for model in models] if models else ['No models available']
        
        # Create widgets
        self.model_selector = widgets.Dropdown(
            options=model_names,
            description='Model:',
            style={'description_width': 'initial'}
        )
        
        self.system_prompt = widgets.Textarea(
            value="You are a helpful assistant.",
            placeholder="Enter system prompt (optional)",
            description='System:',
            layout=widgets.Layout(width='100%', height='60px'),
            style={'description_width': 'initial'}
        )
        
        self.user_input = widgets.Textarea(
            placeholder="Type your message here...",
            description='Message:',
            layout=widgets.Layout(width='100%', height='80px'),
            style={'description_width': 'initial'}
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='primary',
            icon='paper-plane'
        )
        
        self.clear_button = widgets.Button(
            description='Clear History',
            button_style='warning',
            icon='trash'
        )
        
        self.output_area = widgets.Output()
        
        # Set up event handlers
        self.send_button.on_click(self.send_message)
        self.clear_button.on_click(self.clear_history)
        
    def send_message(self, b):
        if not self.user_input.value.strip():
            return
        
        with self.output_area:
            model = self.model_selector.value
            system = self.system_prompt.value if self.system_prompt.value.strip() else None
            prompt = self.user_input.value
            
            print(f"\n{'='*60}")
            print(f"👤 You: {prompt}")
            print(f"🤖 {model}: ", end="")
            
            start_time = time.time()
            response = ollama.generate(model, prompt, system=system)
            end_time = time.time()
            
            if 'error' in response:
                print(f"❌ Error: {response['error']}")
            else:
                assistant_response = response.get('response', 'No response')
                print(assistant_response)
                
                # Store in history
                self.conversation_history.append({
                    'timestamp': datetime.now(),
                    'model': model,
                    'system': system,
                    'prompt': prompt,
                    'response': assistant_response,
                    'duration': end_time - start_time
                })
            
            print(f"\n⏱️ Response time: {end_time - start_time:.2f}s")
        
        # Clear input
        self.user_input.value = ""
    
    def clear_history(self, b):
        self.conversation_history = []
        self.output_area.clear_output()
        with self.output_area:
            print("Conversation history cleared.")
    
    def display(self):
        return widgets.VBox([
            widgets.HTML("<h3>🤖 Interactive Ollama Chat</h3>"),
            self.model_selector,
            self.system_prompt,
            self.user_input,
            widgets.HBox([self.send_button, self.clear_button]),
            self.output_area
        ])

# Create and display the chat interface
chat = InteractiveChat()
display(chat.display())

## 6. Parameter Experimentation

Let's explore how different parameters affect the model's responses.

In [ ]:
# Parameter experimentation interface
def create_parameter_experiment():
    """Create an interface to experiment with different parameters"""
    
    # Get available models
    models = ollama.list_models()
    model_names = [model['name'] for model in models] if models else ['No models available']
    
    # Create widgets for parameters
    model_widget = widgets.Dropdown(
        options=model_names,
        description='Model:'
    )
    
    prompt_widget = widgets.Textarea(
        value="Write a short story about a robot learning to paint.",
        description='Prompt:',
        layout=widgets.Layout(width='100%', height='80px')
    )
    
    system_prompts = {
        'Default': "You are a helpful assistant.",
        'Creative Writer': "You are a creative writer who loves crafting imaginative stories with vivid descriptions.",
        'Technical Expert': "You are a technical expert who provides precise, detailed explanations.",
        'Casual Friend': "You are a casual, friendly person who speaks in a relaxed, conversational tone."
    }
    
    system_widget = widgets.Dropdown(
        options=list(system_prompts.keys()),
        description='Style:'
    )
    
    run_button = widgets.Button(
        description='Generate Responses',
        button_style='success',
        icon='play'
    )
    
    output_area = widgets.Output()
    
    def run_experiment(b):
        with output_area:
            clear_output()
            
            model = model_widget.value
            prompt = prompt_widget.value
            system_key = system_widget.value
            system = system_prompts[system_key]
            
            print(f"🧪 Experiment: {system_key} Style")
            print(f"📝 Prompt: {prompt}")
            print(f"🤖 Model: {model}")
            print("\n" + "="*60 + "\n")
            
            start_time = time.time()
            response = ollama.generate(model, prompt, system=system)
            end_time = time.time()
            
            if 'error' in response:
                print(f"❌ Error: {response['error']}")
            else:
                print(response.get('response', 'No response'))
                print(f"\n⏱️ Generated in {end_time - start_time:.2f} seconds")
    
    run_button.on_click(run_experiment)
    
    return widgets.VBox([
        widgets.HTML("<h3>🧪 Parameter Experimentation</h3>"),
        widgets.HTML("<p>Try different system prompts to see how they affect the model's responses:</p>"),
        model_widget,
        prompt_widget,
        system_widget,
        run_button,
        output_area
    ])

display(create_parameter_experiment())

## 7. Performance Analysis

Let's analyze the performance of different models and track response times.

In [ ]:
# Performance analysis
def analyze_performance():
    """Analyze performance across different models and prompts"""
    
    models = ollama.list_models()
    if not models:
        print("No models available for performance analysis.")
        return
    
    test_prompts = [
        "Hello, how are you?",
        "Explain quantum computing in simple terms.",
        "Write a haiku about programming."
    ]
    
    results = []
    
    print("🔍 Running performance analysis...\n")
    
    for model in models[:2]:  # Test first 2 models to save time
        model_name = model['name']
        print(f"Testing {model_name}...")
        
        for i, prompt in enumerate(test_prompts, 1):
            print(f"  Prompt {i}/3: ", end="")
            
            start_time = time.time()
            response = ollama.generate(model_name, prompt)
            end_time = time.time()
            
            if 'error' not in response:
                duration = end_time - start_time
                response_text = response.get('response', '')
                word_count = len(response_text.split())
                
                results.append({
                    'Model': model_name,
                    'Prompt': f"Prompt {i}",
                    'Duration (s)': duration,
                    'Words': word_count,
                    'Words/sec': word_count / duration if duration > 0 else 0
                })
                
                print(f"{duration:.2f}s ({word_count} words)")
            else:
                print("Error")
        
        print()
    
    # Create DataFrame and visualize results
    if results:
        df = pd.DataFrame(results)
        
        # Display summary table
        print("📊 Performance Summary:")
        print(df.to_string(index=False))
        
        # Create visualizations
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Response time comparison
        df.groupby('Model')['Duration (s)'].mean().plot(kind='bar', ax=ax1, color='skyblue')
        ax1.set_title('Average Response Time by Model')
        ax1.set_ylabel('Seconds')
        ax1.tick_params(axis='x', rotation=45)
        
        # Words per second comparison
        df.groupby('Model')['Words/sec'].mean().plot(kind='bar', ax=ax2, color='lightgreen')
        ax2.set_title('Average Words per Second by Model')
        ax2.set_ylabel('Words/sec')
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        return df
    else:
        print("No successful responses to analyze.")
        return None

# Run performance analysis
performance_data = analyze_performance()

## 8. Conversation History Analysis

Let's analyze the conversation history from our interactive chat.

In [ ]:
# Analyze conversation history
def analyze_conversation_history(chat_instance):
    """Analyze the conversation history from the interactive chat"""
    
    if not chat_instance.conversation_history:
        print("No conversation history to analyze. Try using the interactive chat above first!")
        return
    
    history = chat_instance.conversation_history
    
    print(f"📈 Conversation Analysis ({len(history)} interactions)\n")
    
    # Basic statistics
    total_duration = sum(conv['duration'] for conv in history)
    avg_duration = total_duration / len(history)
    
    print(f"⏱️ Total conversation time: {total_duration:.2f} seconds")
    print(f"⏱️ Average response time: {avg_duration:.2f} seconds")
    
    # Model usage
    models_used = {}
    for conv in history:
        model = conv['model']
        models_used[model] = models_used.get(model, 0) + 1
    
    print(f"\n🤖 Models used:")
    for model, count in models_used.items():
        print(f"  {model}: {count} times")
    
    # Response length analysis
    response_lengths = [len(conv['response'].split()) for conv in history]
    avg_length = sum(response_lengths) / len(response_lengths)
    
    print(f"\n📝 Response analysis:")
    print(f"  Average response length: {avg_length:.1f} words")
    print(f"  Shortest response: {min(response_lengths)} words")
    print(f"  Longest response: {max(response_lengths)} words")
    
    # Create timeline visualization
    if len(history) > 1:
        timestamps = [conv['timestamp'] for conv in history]
        durations = [conv['duration'] for conv in history]
        
        plt.figure(figsize=(10, 4))
        plt.plot(range(1, len(durations) + 1), durations, 'o-', color='blue', alpha=0.7)
        plt.title('Response Time Over Conversation')
        plt.xlabel('Interaction Number')
        plt.ylabel('Response Time (seconds)')
        plt.grid(True, alpha=0.3)
        plt.show()

# Analyze the chat history (if available)
try:
    analyze_conversation_history(chat)
except NameError:
    print("Chat instance not available. Use the interactive chat above to generate some conversation history first!")

## 9. Next Steps and Resources

Congratulations! You've learned the basics of using Ollama for local LLM deployment. Here's what you can explore next:

In [ ]:
# Next steps and additional resources
print("🎉 Congratulations on completing the Ollama introduction!")
print("\n📚 Next notebooks to explore:")
print("  • 02_transformers_basics.ipynb - Alternative approaches")
print("  • 03_model_formats_explained.ipynb - Model formats")
print("  • 04_prompt_engineering.ipynb - Advanced prompting")
print("  • 05_performance_optimization.ipynb - Optimization")
print("\n🔗 External resources:")
print("  • Ollama Documentation: https://ollama.ai/docs")
print("  • Model Library: https://ollama.ai/library")
print("  • Community: https://github.com/ollama/ollama")
print("\n🚀 Happy experimenting with local LLMs!")